### Fundamentos

Los Spark DataFrames son la piedra angular y la forma principal de trabajar con Spark y Python después de Spark 2.0.

Los DataFrames actúan como potentes facilitadores para el manejo de tablas, y manejan fácilmente grandes conjuntos de datos.

El cambio a DataFrames ofrece muchas ventajas:
* Una sintaxis mucho más simple
* Capacidad de usar SQL directamente en el marco de datos
* Las operaciones se distribuyen automáticamente entre los RDD

Si ha usado anteriormente la biblioteca de pandas de Python o incluso R, probablemente ya esté familiarizado con el concepto de DataFrames.

Los Spark DataFrames amplían muchos de estos conceptos, lo que le permite transferir ese conocimiento fácilmente al comprender la sintáxis simple.

Recuerde que la principal ventaja de usar Spark DataFrames frente a otras herramientas es que Spark puede manejar grandes conjuntos de datos que no cabrían en una sola computadora.

In [0]:
dbutils.fs.ls("/")

In [0]:
generation = "mod4gen13"

Para montar un Blob Storage en Azure Databricks:
```
dbutils.fs.mount(
  source = "wasbs://<container-name>@<storage-account-name>.blob.core.windows.net",
  mount_point = "/mnt/<mount-name>",
  extra_configs = {"fs.azure.account.key.<storage-account-name>.blob.core.windows.net": "<your-key>"})
```

In [0]:
dbutils.fs.mount(
  source = "wasbs://input@minidatalake.blob.core.windows.net",
  mount_point = f"/mnt/{generation}/input",
  extra_configs = {"fs.azure.account.key.minidatalake.blob.core.windows.net": "<your-key>"})

In [0]:
dbutils.fs.mount(
  source = "wasbs://output@minidatalake.blob.core.windows.net",
  mount_point = f"/mnt/{generation}/output",
  extra_configs = {"fs.azure.account.key.minidatalake.blob.core.windows.net": "<your-key>"})

In [0]:
# dbutils.fs.unmount(f"/mnt/{generation}/input")
# dbutils.fs.unmount(f"/mnt/{generation}/output")

In [0]:
dbutils.fs.ls(f"/mnt/{generation}/input")

In [0]:
df = spark.read.csv(f"/mnt/{generation}/input/appl_stock.csv", inferSchema=True, header=True)

In [0]:
df.show()

In [0]:
df.display()

Carga de datos semi-estructurados

In [0]:
df = spark.read.json(f"/mnt/{generation}/input/people.json")

In [0]:
# Observe cómo faltan datos
display(df)

In [0]:
df.printSchema()

In [0]:
df.columns

In [0]:
df.describe().display()

Algunos tipos de datos facilitan la inferencia del esquema (como formatos tabulares como CSV).

Sin embargo, a menudo se tiene que configurar un esquema manualmente si se está trabajando con un método `.read` que no tiene `inferSchema()` incorporado.

Spark tiene algunas funciones necesarias para realizar esta operación, solo se requiere una estructura más específica.

In [0]:
from pyspark.sql.types import StructField, StringType, IntegerType, StructType

In [0]:
data_schema = [StructField("age", IntegerType(), True),StructField("name", StringType(), True)]

In [0]:
final_struc = StructType(fields=data_schema)

In [0]:
df = spark.read.json(f"/mnt/{generation}/input/people.json", schema=final_struc)

In [0]:
df.printSchema()

In [0]:
df['age']

In [0]:
type(df['age'])

In [0]:
df.select('age')

In [0]:
type(df.select('age'))

In [0]:
df.select('age').display()

In [0]:
# Devuelve la lista de objetos Row
df.head(2)

Múltiples columnas:

In [0]:
df.select(['age','name'])

In [0]:
df.select(['age','name']).display()

Creación de nuevas columnas

In [0]:
# Agregar una nueva columna con una copia simple
df.withColumn('newage', df['age']).display()

In [0]:
df.display()

In [0]:
# Renombrado simple
df.withColumnRenamed('age','supernewage').display()

Operaciones más complejas para crear nuevas columnas.

In [0]:
df.withColumn('doubleage', df['age']*2).display()

In [0]:
df.withColumn('add_one_age', df['age']+1).display()

In [0]:
df.withColumn('half_age', df['age']/2).display()

In [0]:
df.withColumn('half_age',df['age']/2)

### Uso de SQL

Para usar consultas SQL directamente con el dataframe, se debe registrar en una vista temporal

In [0]:
# Registra el DataFrame como una vista temporal de SQL
df.createOrReplaceTempView("people")

In [0]:
sql_results = spark.sql("SELECT * FROM people")

In [0]:
sql_results

In [0]:
sql_results.display()

In [0]:
spark.sql("SELECT * FROM people WHERE age=30").display()